In [1]:
VERSION = '99_v1_02_15'

In [2]:
model_list = [    
    #{'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':0.5}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':0.5}, #deberta-base, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':0.5}, #longformer-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_07/oof_df.csv', 'w':1}, #deberta-large, seed200, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v1_04/oof_df.csv', 'w':1}, #deberta-v3-large, seed200, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v1_05/oof_df.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
    pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
    pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5876


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,...,Effective_3,Ineffective_4,Adequate_4,Effective_4,Ineffective_5,Adequate_5,Effective_5,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.002373,0.166887,0.012559,1,0.085691,...,0.004693,0.003977,0.173669,0.004172,0.009581,0.164896,0.007341,0.027909,0.919085,0.053007
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.010181,0.166613,0.005024,1,0.087332,...,0.003621,0.040623,0.139511,0.001684,0.020762,0.156338,0.004718,0.099840,0.876656,0.023504
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.025583,0.154209,0.002026,1,0.164698,...,0.001394,0.052475,0.128451,0.000892,0.021628,0.156904,0.003286,0.202142,0.785135,0.012722
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.060800,0.119449,0.001570,1,0.420119,...,0.002481,0.029571,0.150564,0.001683,0.033277,0.146019,0.002522,0.237601,0.749702,0.012697
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.045044,0.136008,0.000767,1,0.290297,...,0.001050,0.040517,0.140495,0.000806,0.061235,0.118693,0.001890,0.296388,0.695423,0.008189
